In [1]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm 

# Don't collapse Pandas Dataframes:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def normalize(data): ## Normalize Data between 0 and 1
    min_val = min(data)
    if min_val < 0:
        data = [x + abs(min_val) for x in data]
    max_val = max(data)
    normalized_data = [x/max_val for x in data]
    return normalized_data
  
def one_hot_encoding_column(df, column):
    ohe_df = pd.get_dummies(df[column])
    ohe_df.reset_index(drop = True, inplace = True)
    df = pd.concat([df, ohe_df], axis = 1)
    return df

def cosine_sim(v1,v2):
    cosine_similarity = sum(dot(v1,v2)/(norm(v1)*norm(v2)))
    return cosine_similarity

def recommend(df, book_id, number_of_recommendations):
    inputVec = df.loc[book_id].values
    df['Cosine Similarity']= df.apply(lambda x: cosine_sim(inputVec, x.values), axis=1)
    df = df.nlargest(columns='Cosine Similarity', n=number_of_recommendations) # returns top n user specified books
    return df


In [3]:
## Import Dataset
df = pd.read_csv('dataset.csv')
df['num_pages_norm'] = normalize(df['num_pages'].values)
df['book_rating_norm'] = normalize(df['book_rating'].values)
df['book_price_norm'] = normalize(df['book_price'].values)
print('Dataset Shape: {0}'.format(df.shape))
df.sort_values(['user_id'], ascending=False).head(20)

Dataset Shape: (100000, 13)


,book_id,author_id,book_genre,user_id,num_pages,book_rating,publisher_id,publish_year,book_price,text_lang,num_pages_norm,book_rating_norm,book_price_norm
29172,171,427,6,30000,566,2,6,2020,132,3,0.808571,0.2,0.660
62781,646,370,3,30000,475,8,27,2006,114,6,0.678571,0.8,0.570
68263,402,64,9,30000,349,10,17,2001,74,5,0.498571,1.0,0.370
36586,1750,90,2,29999,414,8,33,2009,126,6,0.591429,0.8,0.630
64164,664,359,7,29999,492,10,23,2011,173,3,0.702857,1.0,0.865
61097,1541,301,7,29999,631,7,31,2005,115,3,0.901429,0.7,0.575
96053,354,181,4,29999,91,6,44,2011,16,6,0.130000,0.6,0.080
3363,1679,62,2,29998,93,4,19,2009,93,3,0.132857,0.4,0.465
9513,1320,365,5,29998,354,9,1,2000,178,7,0.505714,0.9,0.890
13174,2229,95,3,29998,514,5,18,2021,176,1,0.734286,0.5,0.880


In [4]:
df = one_hot_encoding_column(df = df, column = 'publish_year') # Feature & Labels
df = one_hot_encoding_column(df = df, column = 'book_genre') # Feature & Labels
df = one_hot_encoding_column(df = df, column = 'text_lang') # Feature & Labels
df.drop(columns = ['publish_year', 'book_genre', 'num_pages', 'book_rating', 'book_price', 'text_lang'], inplace = True) # Remove Columns that aren't Features
df.set_index('book_id', inplace = True) # Set Instance as Index
df.sort_values(['book_id'], ascending=False).head(20)

,author_id,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3000,139,29561,33,0.684286,0.8,0.460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3000,213,1954,42,0.227143,0.5,0.795,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3000,402,4738,50,0.538571,0.4,0.130,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3000,7,1794,2,0.207143,0.5,0.705,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
3000,313,10960,48,0.500000,0.6,0.390,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3000,187,12893,2,0.948571,0.6,0.360,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
3000,39,12174,16,0.987143,0.2,0.405,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3000,376,1928,27,0.470000,0.8,0.425,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3000,282,1627,40,0.124286,1.0,0.395,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [5]:
# ran on a sample as an example
book_id = 1001
number_of_recommendations = 5
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df.head(10)

,author_id,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,Cosine Similarity
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1318,431,25294,44,0.562857,0.9,0.930,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,4.792067
464,369,21690,38,0.867143,0.4,0.930,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4.792067
2514,359,21073,37,0.797143,0.3,0.315,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.792067
2160,369,21636,38,0.308571,0.7,0.395,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,4.792067
924,449,26262,45,0.395714,0.9,0.345,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,4.792067


In [6]:
# ran on a sample as an example
book_id = 1318
number_of_recommendations = 5
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df.head(10)

,author_id,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,Cosine Similarity
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
700,271,16494,28,0.125714,0.6,0.365,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,5.633985
1849,296,17972,29,0.485714,0.3,0.650,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,5.633985
319,297,18073,31,0.274286,0.1,0.885,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,5.633985
1092,248,15096,25,0.532857,1.0,0.460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,5.633985
1558,325,19751,34,0.492857,0.6,0.155,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,5.633985


In [7]:
# ran on a sample as an example
book_id = 700
number_of_recommendations = 5
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df.head(10)

,author_id,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,Cosine Similarity
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1800,302,16386,31,0.580000,0.2,0.050,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,4.553203
240,314,17069,32,0.697143,0.4,0.340,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,4.553203
1488,284,15358,29,0.761429,0.8,0.640,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,4.553203
2925,270,14665,27,0.274286,0.9,0.945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,4.553203
1494,254,13798,27,0.908571,0.4,0.110,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,4.553203


In [8]:
# ran on a sample as an example
book_id = 302
number_of_recommendations = 5
recommendation_df = recommend(df, book_id, number_of_recommendations)
recommendation_df.head(10)

,author_id,user_id,publisher_id,num_pages_norm,book_rating_norm,book_price_norm,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,Cosine Similarity
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1794,293,16638,33,0.924286,0.6,0.355,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,5.55358
106,295,16755,33,0.438571,0.8,0.425,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,5.55358
250,254,14399,28,0.238571,1.0,0.210,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,5.55358
716,296,16783,33,0.195714,0.5,0.450,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,5.55358
2589,292,16533,31,0.927143,0.5,0.455,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,5.55358
